In [ ]:
# !unzip "/content/drive/MyDrive/Đồ án DS201 - Deep Learning/Data/Data train model/Zip/BiPS.zip" 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
path_of_cd="/content/drive/MyDrive/DOAN_DL/"
# path_of_dataset='/content/drive/MyDrive/DOAN_DLt/BiPS/'
# path_of_dataset_img='/content/drive/MyDrive/Deep-Learning/DOAN_DL/content/BiPS_img/'

In [ ]:
# %cd '/content/content/BiPS/'
# import shutil

# shutil.rmtree('/content/content/BiPS/train/Other')
# shutil.rmtree('/content/content/BiPS/test/Other')
# shutil.rmtree('/content/content/BiPS/val/Other')

/content/content/BiPS


In [ ]:
#EXTRACTING FRAMES

import csv
import glob
import os.path
from subprocess import call
import os

def extract_files():
    data_file = []
    folders = ['train','test','val']
    %cd '/content/drive/MyDrive/DOAN_DL/'
    for folder in folders:
        class_folders = glob.glob(os.path.join(folder, '*'))
        print(folder)
        for vid_class in class_folders:
            class_files = glob.glob(os.path.join(vid_class, '*.mp4'))
            print(vid_class)
            for video_path in class_files:
                # Get the parts of the file.
                print(video_path)
                video_parts = get_video_parts(video_path)

                train_or_test_or_val, classname, filename_no_ext, filename = video_parts

                # Only extract if we haven't done it yet. Otherwise, just get
                # the info.
                if not check_already_extracted(video_parts):
                    # Now extract it.
                    src = os.path.join(train_or_test_or_val, classname, filename)
                    dest = os.path.join(train_or_test_or_val, classname,
                        filename_no_ext + '-%04d.jpg')
                    call(["ffmpeg", "-i" ,src,'-r', '1' ,dest])

                # Now get how many frames it is.
                nb_frames = get_nb_frames_for_video(video_parts)

                data_file.append([train_or_test_or_val, classname, filename_no_ext, nb_frames])

                #print("Generated %d frames for %s" % (nb_frames, filename_no_ext))

    with open(path_of_cd+'data_file.csv', 'w', newline='') as fout:
        writer = csv.writer(fout)
        writer.writerows(data_file)
    #[train|test], class, filename, nb frames   
    print("Extracted and wrote %d video files." % (len(data_file)))

def get_nb_frames_for_video(video_parts):
    train_or_test_or_val, classname, filename_no_ext, _ = video_parts
    generated_files = glob.glob(os.path.join(train_or_test_or_val, classname,
                                filename_no_ext + '*.jpg'))
    return len(generated_files)

def get_video_parts(video_path):
    parts = video_path.split(os.path.sep)
    filename = parts[2]
    filename_no_ext = filename.split('.')[0]
    classname = parts[1]
    train_or_test_or_val = parts[0]

    return train_or_test_or_val, classname, filename_no_ext, filename

def check_already_extracted(video_parts):
    train_or_test_or_val, classname, filename_no_ext, _ = video_parts
    return bool(os.path.exists(os.path.join(train_or_test_or_val, classname,
                               filename_no_ext + '-0001.jpg')))

extract_files()


In [ ]:
# import os
# %cd path_of_dataset
# folders = ['train','test','val']
# for folder in folders:
#      class_folders = glob.glob(os.path.join(folder, '*'))
#      for class_folder in class_folders:
#        test = os.listdir(class_folder)
#        for item in test:
#           if item.endswith(".jpg"):
#             os.remove(os.path.join(class_folder, item))

In [ ]:
import gc
gc.collect()

3

In [ ]:
%cd "/content/drive/MyDrive/DOAN_DL/"

/content/drive/.shortcut-targets-by-id/12Wt3p74m-tMJ2wPK2ubma_Ql17Wt6_CR/DOAN_DL


In [ ]:
!pwd

/content/drive/.shortcut-targets-by-id/12Wt3p74m-tMJ2wPK2ubma_Ql17Wt6_CR/DOAN_DL


In [ ]:
import csv
import numpy as np
import random
import glob
import os.path
import sys
import operator
import threading

from keras.utils import to_categorical
class DataSet():

    def __init__(self, seq_length=40, class_limit=None, image_shape=(224, 224, 3)):
        self.seq_length = seq_length
        self.class_limit = class_limit
        self.sequence_path = os.path.join('sequences')
        self.max_frames = 300  # max number of frames a video can have for us to use it
        self.data = self.get_data()
        self.classes = self.get_classes()
        self.data = self.clean_data()
        self.image_shape = image_shape

    @staticmethod
    def get_data():
        with open(os.path.join('data_file.csv'), 'r') as fin:
            reader = csv.reader(fin)
            data = list(reader)
        return data

    def clean_data(self):
        data_clean = []
        for item in self.data:
            if int(item[3]) <= self.max_frames \
                    and item[1] in self.classes:
                data_clean.append(item)

        return data_clean

    def get_classes(self):
        classes = []
        for item in self.data:
            if item[1] not in classes:
                classes.append(item[1])
        classes = sorted(classes)
        if self.class_limit is not None:
            return classes[:self.class_limit]
        else:
            return classes

    def get_class_one_hot(self, class_str):
        # Encode it first.
        label_encoded = self.classes.index(class_str)
        # Now one-hot it.
        label_hot = to_categorical(label_encoded, len(self.classes))
        assert len(label_hot) == len(self.classes)
        return label_hot

    def split_train_test_val(self):
        train = []
        test = []
        val = []
        for item in self.data:
            if item[0] == 'train':
                train.append(item)
            elif item[0] == 'val':
                val.append(item)
            elif item[0] == 'test':
                test.append(item)
        return train, test ,val

    def get_all_sequences_in_memory(self, train_test_val, data_type):
        train, test ,val = self.split_train_test_val()
        if train_test_val == 'train' :
          data = train
        elif train_test_val == 'test' :
          data = test
        elif train_test_val == 'val':  
          data = val

        print("Loading %d samples into memory for %sing." % (len(data), train_test_val))

        X, y = [], []
        for row in data:
            sequence = self.get_extracted_sequence(data_type, row)
            if sequence is None:
                print("Can't find sequence. Did you generate them?")
                raise
            X.append(sequence)
            y.append(self.get_class_one_hot(row[1]))
        return np.array(X), np.array(y)

    def get_extracted_sequence(self, data_type, sample):
        filename = sample[2]
        path = os.path.join(self.sequence_path, filename + '-' + str(self.seq_length) + \
            '-' + data_type + '.npy')
        if os.path.isfile(path):
            return np.load(path)
        else:
            return None

    def get_frames_by_filename(self, filename, data_type):
        sample = None
        for row in self.data:
            if row[2] == filename:
                sample = row
                break
        if sample is None:
            raise ValueError("Couldn't find sample: %s" % filename)
        sequence = self.get_extracted_sequence(data_type, sample)
        if sequence is None:
            raise ValueError("Can't find sequence. Did you generate them?")
        return sequence

    @staticmethod
    def get_frames_for_sample(sample):
        """Given a sample row from the data file, get all the corresponding frame
        filenames."""
        path = os.path.join(sample[0], sample[1])
        filename = sample[2]
        images = sorted(glob.glob(os.path.join(path, filename + '*jpg')))
        return images

    @staticmethod
    def rescale_list(input_list, size):
        assert len(input_list) >= size
        skip = len(input_list) // size
        output = [input_list[i] for i in range(0, len(input_list), skip)]
        return output[:size]

In [ ]:
import gc
gc.collect()

0

In [ ]:
import numpy as np
import os.path
import tensorflow as tf
from keras.utils import load_img, img_to_array
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
from tqdm import tqdm


# Get the dataset.
data = DataSet(seq_length=40, class_limit=70)
seq_length=40
base_model = InceptionV3(
    weights='imagenet',
    include_top=True
)
# We'll extract features at the final pool layer.
model = Model(
    inputs=base_model.input,
    outputs=base_model.get_layer('avg_pool').output
)

# Loop through data.
pbar = tqdm(total=len(data.data))
for video in data.data:

    # Get the path to the sequence for this video.
    path = os.path.join('sequences', video[2] + '-' + str(seq_length) + \
        '-features.npy')  # numpy will auto-append .npy
    # Check if we already have it.
    if os.path.isfile(path + '.npy'):
        pbar.update(1)
        continue

    # Get the frames for this video.
    frames = data.get_frames_for_sample(video)
    #print(frames)

    # Now downsample to just the ones we need.
    frames = data.rescale_list(frames, 3)
    #print(frames)
    #extracting features and appending to build the sequence.
    sequence = []
    for image in frames:
        img = load_img(image, target_size=(299, 299))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = model.predict(x)
        sequence.append(features[0])

    # Save the sequence.
    np.save(path, sequence)
    print('\nsave',path )

    pbar.update(1)

pbar.close()


 18%|█▊        | 651/3559 [40:00<2:58:42,  3.69s/it]


1/1 [==============================] - 0s 23ms/step



  0%|          | 1/3559 [00:01<1:52:39,  1.90s/it]


save sequences/video_1159-40-features.npy
1/1 [==============================] - 0s 23ms/step



  0%|          | 2/3559 [00:03<1:25:19,  1.44s/it]

...


save sequences/video_1247-40-features.npy
1/1 [==============================] - 0s 23ms/step



100%|█████████▉| 3558/3559 [1:56:07<00:02,  2.05s/it]


save sequences/video_3293-40-features.npy
1/1 [==============================] - 0s 24ms/step



100%|██████████| 3559/3559 [1:56:10<00:00,  1.96s/it]


save sequences/video_4949-40-features.npy


In [ ]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from collections import deque
import sys
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
import os.path
checkpointer = ModelCheckpoint(
    filepath=os.path.join( 'checkpoints','lstm-features' + '.{epoch:03d}-{val_loss:.3f}.hdf5'),
    verbose=1,
    save_best_only=True)

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join( 'logs', 'lstm'))

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=10)

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join( 'logs', 'lstm' + '-' + 'training-' + \
    str(timestamp) + '.log'))

# Get the data and process it.
data = DataSet(
    seq_length=40,
    class_limit=70
)
#listt=[]
#listt2=[]
X, y = data.get_all_sequences_in_memory('train', 'features')

X_val, y_val = data.get_all_sequences_in_memory('val', 'features')
# X_test, y_test = data.get_all_sequences_in_memory('test', 'features')


# for i in range(len(X)):
#  for j in range(70):
  #   if (y[i][j]==1) and not(j in listt):
  #    listt.append(j)
# for i in range(len(X_test)):
#  for j in range(70):
  #   if (y_test[i][j]==1) and not(j in listt2):
  #    listt2.append(j)
#print(listt)
#print(listt2)
#listt3= []
# for i in range(len(listt2)):
#  if not(listt2[i] in listt):
  #   listt3.append(listt2[i])
#X_test2 = X_test.copy()
#y_test2 = y_test.copy()
#for i in range(len(X_test)):
  # flag=1
  #for j in range(70):
    # if(y_test[i][j]==1) and (j in listt3):
    #  flag=0
      # break
  #if flag==1:
    # X_test2 = np.append(X_test2,[X_test[i]],axis=0)
    #y_test2 = np.append(y_test2,[y_test[i]],axis=0)
#print(X_test2.shape)
#print(y_test2.shape)
#l = X_test2.shape[0]-X_test.shape[0]
#X_test = X_test2[-l:,:,:]
#y_test = y_test2[-l:,:]

model = Sequential()
model.add(LSTM(2048, return_sequences=False,input_shape=(3,2048),dropout=0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(data.classes), activation='softmax'))
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                    metrics=['accuracy','top_k_categorical_accuracy'])
print(model.summary())

model.fit(
    X,
    y,
    batch_size=32,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[tb, csv_logger,checkpointer],
    epochs=100)
    

Loading 2845 samples into memory for training.
Loading 355 samples into memory for valing.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 2048)              33562624  
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 34,615,816
Trainable params: 34,615,816
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
87/89 [============================>.] - ETA: 0s - loss: 1.7842 - 

In [ ]:
# Evaluate the model on the test data using `evaluate`
X_test, y_test = data.get_all_sequences_in_memory('test', 'features')
print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

Loading 359 samples into memory for testing.
Evaluate on test data
12/12 [==============================] - 0s 13ms/step - loss: 1.2916 - accuracy: 0.5571 - top_k_categorical_accuracy: 0.9666
test loss, test acc: [1.291616678237915, 0.5571030378341675, 0.9665738344192505]


In [ ]:
y_pred=model.predict(X_test)

12/12 [==============================] - 0s 15ms/step


In [ ]:
y_test_classes = y_test.argmax(1)
y_pred_classes = y_pred.argmax(1)


In [ ]:
y_pred

array([[0.00149851, 0.00786755, 0.01511407, ..., 0.48501086, 0.18833488,
        0.12524107],
       [0.0290716 , 0.12041166, 0.16426654, ..., 0.1892567 , 0.24799033,
        0.01957598],
       [0.14397068, 0.2171377 , 0.05307908, ..., 0.3436721 , 0.0402601 ,
        0.03145074],
       ...,
       [0.00090744, 0.15197273, 0.01565658, ..., 0.05894297, 0.09376743,
        0.00529828],
       [0.0026164 , 0.15342818, 0.04186108, ..., 0.6266619 , 0.10506444,
        0.00257626],
       [0.02125504, 0.0073526 , 0.01152496, ..., 0.05074086, 0.87875056,
        0.00282498]], dtype=float32)

In [ ]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
data.get_classes()

NameError: ignored

In [ ]:
target_names =data.get_classes()

In [ ]:
print(classification_report(y_test_classes, y_pred_classes,target_names=target_names))

              precision    recall  f1-score   support

       Bring       0.73      0.72      0.72        53
       Chase       0.40      0.33      0.36        30
        Fall       0.00      0.00      0.00        14
       Fight       0.29      0.12      0.17        33
    NonSleep       0.78      0.98      0.87        44
      Normal       0.52      0.80      0.63       120
       Other       0.39      0.19      0.25        48
   OutOfSeat       0.00      0.00      0.00        17

    accuracy                           0.56       359
   macro avg       0.39      0.39      0.38       359
weighted avg       0.49      0.56      0.50       359



In [ ]:
data = DataSet(seq_length=40, class_limit=70)

In [ ]:
data.get_classes()

['Bring', 'Chase', 'Fall', 'Fight', 'NonSleep', 'Normal', 'Other', 'OutOfSeat']

In [ ]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from collections import deque
import sys
import time
import os.path
import glob

model = Sequential()
model.add(LSTM(2048, return_sequences=False,input_shape=(3,2048),dropout=0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.load_weights('/content/drive/MyDrive/DOAN_DL/checkpoints/lstm-features.037-1.244.hdf5')
print(model.summary())

classes = data.get_classes()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 2048)              33562624  
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 8)                 4104      
                                                                 
Total params: 34,615,816
Trainable params: 34,615,816
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_test, y_test = data.get_all_sequences_in_memory('test', 'features')
y_pred=model.predict(X_test)

Loading 359 samples into memory for testing.
12/12 [==============================] - 5s 10ms/step


In [ ]:
y_test_classes = y_test.argmax(1)
y_pred_classes = y_pred.argmax(1)


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test_classes, y_pred_classes,target_names=classes,digits=4))


              precision    recall  f1-score   support

       Bring     0.6724    0.7358    0.7027        53
       Chase     0.4706    0.2667    0.3404        30
        Fall     0.0000    0.0000    0.0000        14
       Fight     0.3333    0.1212    0.1778        33
    NonSleep     0.7925    0.9545    0.8660        44
      Normal     0.5026    0.7917    0.6149       120
       Other     0.3462    0.1875    0.2432        48
   OutOfSeat     0.0000    0.0000    0.0000        17

    accuracy                         0.5487       359
   macro avg     0.3897    0.3822    0.3681       359
weighted avg     0.4807    0.5487    0.4927       359



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save('/content/drive/MyDrive/DOAN_DL/mymodel.h5')

To run on a particular kind of videos provided in UCF101:

In [ ]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from collections import deque
import sys
import time
import os.path
import glob

model = Sequential()
model.add(LSTM(2048, return_sequences=False,input_shape=(3,2048),dropout=0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(70, activation='softmax'))
model.load_weights('data/checkpoints/lstm-features.025-1.006.hdf5')
print(model.summary())

classes = glob.glob("data/train/*")
classes = [classes[i].split('/')[2] for i in range(len(classes))]
classes = sorted(classes)

In [ ]:
import glob
import numpy as np
import os.path
from keras.preprocessing import image as Img
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
from tqdm import tqdm

path= 'UCF101/BoxingSpeedBag/'
y = glob.glob(path+'*.avi')
for Z in y: 
  name = (Z.split('/')[2]).split('.')[0]
  name = name+'-40-features.npy'
  X = glob.glob("data/sequences/"+name)
  seq = np.array([[]])
  flag=1
  for i in X:
    tmp = np.load(i)
    if flag==1:
      seq = tmp
    else:
      seq = np.append(seq,tmp,0)
  seq = np.array([seq])
  if(seq.shape != (1,40,2048)):
    continue
  prediction = model.predict(seq)

  maxm = prediction[0][0]
  maxid = 0
  for i in range(len(prediction[0])):
    if(maxm<prediction[0][i]):
      maxm = prediction[0][i]
      maxid = i

  print(Z,' ------- ',classes[maxid])

To run the code on a specific video, please upload the video in correct directory and run the cells below:


In [ ]:
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from collections import deque
import sys
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
import time
import os.path

model = Sequential()
model.add(LSTM(2048, return_sequences=False,input_shape=(40,2048),dropout=0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(data.classes), activation='softmax'))
model.load_weights('/content/drive/MyDrive/Deep-Learning/Human-Action-Recognition/checkpoints/lstm-features.023-0.100.hdf5')
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 2048)              33562624  
                                                                 
 dense_6 (Dense)             (None, 512)               1049088   
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 4)                 2052      
                                                                 
Total params: 34,613,764
Trainable params: 34,613,764
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
import os.path
from keras.utils import load_img, img_to_array
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model
from keras.layers import Input
import glob

def rescale_list(input_list, size):
    assert len(input_list) >= size
    skip = len(input_list) // size
    output = [input_list[i] for i in range(0, len(input_list), skip)]
    return output[:size]
classes = glob.glob("train/*")
classes = [classes[i].split('/')[1] for i in range(len(classes))]
classes = sorted(classes)

import cv2 
import os 
image_name_2 ='/content/drive/MyDrive/Deep-Learning/Human-Action-Recognition/test/ApplyLipstick/v_ApplyLipstick_g17_c02.avi'
image_name='v_ApplyLipstick_g17_c02.avi'
cam = cv2.VideoCapture(image_name_2) 
currentframe = 0
  
frames=[]
while(True): 
    ret,frame = cam.read() 
    if ret: 
        # if video is still left continue creating images 
        name = 'testFinal/frame'+image_name +"frame_no"+ str(currentframe) + '.jpg'
        cv2.imwrite(name, frame) 
        frames.append(name)  
        currentframe += 1
    else: 
        break
cam.release() 
cv2.destroyAllWindows() 
rescaled_list = rescale_list(frames,40)

base_model = InceptionV3(
    weights='imagenet',
    include_top=True
)
# We'll extract features at the final pool layer.
inception_model = Model(
    inputs=base_model.input,
    outputs=base_model.get_layer('avg_pool').output
)
sequence = []
for image in rescaled_list:
        img = load_img(image, target_size=(299, 299))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = inception_model.predict(x)
        sequence.append(features[0])

sequence = np.array([sequence])
prediction = model.predict(sequence)
maxm = prediction[0][0]
maxid = 0
for i in range(len(prediction[0])):
  if(maxm<prediction[0][i]):
    maxm = prediction[0][i]
    maxid = i

print(image_name,' ------- ',classes[maxid])

1/1 [==============================] - 0s 341ms/step
v_ApplyLipstick_g17_c02.avi  -------  ApplyLipstick
